<a href="https://colab.research.google.com/github/neuroidss/FreeEEG8-alpha/blob/main/freeeeg8-alpha_create_function_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13320    0 13320    0     0  25158      0 --:--:-- --:--:-- --:--:-- 25179
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True)

In [ ]:
%cd /content
!git clone https://github.com/neuroidss/freeeeg8-alpha
%cd /content/freeeeg8-alpha
!git clone https://github.com/neuroidss/create_function_chat

/content
Cloning into 'freeeeg8-alpha'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 38 (delta 14), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 59.70 KiB | 473.00 KiB/s, done.
Resolving deltas: 100% (14/14), done.
/content/freeeeg8-alpha
Cloning into 'create_function_chat'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 283 (delta 113), reused 48 (delta 48), pack-reused 118 (from 1)
Receiving objects: 100% (283/283), 8.67 MiB | 13.27 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [ ]:
!ollama pull qwen2.5:7b-instruct-q8_0
#!ollama pull llama3.1:8b-instruct-q8_0
#!ollama pull mistral-nemo:12b-instruct-2407-q6_K

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest 
pulling 3603045b543e...   0% ▕▏    0 B/8.1 GB                  pulling manifest 
pulling 3603045b543e...   0% ▕▏    0 B/8.1 GB                  pulling manifest 
pulling 3603045b543e...   0% ▕▏ 444 KB/8.1 GB                  pulling manifest 
pulling 3603045b543e...   0% ▕▏  28 MB/8.1 GB                  pulling manifest 
pulling 3603045b543e...   1% ▕▏  45 MB/8.1 GB                  pulling manifest 
pulling 3603045b543e...   1% ▕▏  82 MB/8.1 GB                  pulling manifest 
pulling 3603045b543e...   1% ▕▏ 104 MB/8.1 GB                  pulling manifest 
pulling 3603045b543e...   2% ▕▏ 134 MB/8.1 GB                  pulling manifest 
pulling 3603045b543e...   2% ▕▏ 173 MB/8.1 GB                  pulli

In [ ]:
!pip install ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install kicad-skip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install git+https://github.com/devbisme/kinet2pcb

  Cloning https://github.com/devbisme/kinet2pcb to /tmp/pip-req-build-4p9a3inp
  Running command git clone --filter=blob:none --quiet https://github.com/devbisme/kinet2pcb /tmp/pip-req-build-4p9a3inp
  Resolved https://github.com/devbisme/kinet2pcb to commit 0257a698ac77c9a133a028fff2b57dc651d4b1e9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.7/313.7 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kinet2pcb: filename=kinet2pcb-1.1.2-py2.py3-none-any.whl size=8530 sha256=9353a5634f75f60db2b2a9527db326c9adc0cadfd4f77101b4cb66719488b8eb
  Stored in directory: /tmp/pip-ephem-wheel-cache-dj_2gmx_/wheels/a4/e2/c1/a2fadf4da33260e27bf8fc9bb1be62642de5c2f52b390ee6b2
  Created wheel for kinparse: filename=kinparse-1.2.1-py2.py3-none-any.whl size=7912 sha256=c8b70e75e6b7c3475605e3b8c4e874d0466

In [ ]:
!add-apt-repository --yes ppa:kicad/kicad-7.0-releases
!apt update
!apt install --install-recommends kicad

Repository: 'deb https://ppa.launchpadcontent.net/kicad/kicad-7.0-releases/ubuntu/ jammy main'
Description:
This PPA will be the Official PPA KiCad 7.0 releases.
More info: https://launchpad.net/~kicad/+archive/ubuntu/kicad-7.0-releases
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/kicad-ubuntu-kicad-7_0-releases-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/kicad-ubuntu-kicad-7_0-releases-jammy.list
Adding key to /etc/apt/trusted.gpg.d/kicad-ubuntu-kicad-7_0-releases.gpg with fingerprint FDA854F61C4D0D9572BB95E5245D5502FAD7A805
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InReleas

In [ ]:
%cd /content/freeeeg8-alpha
!python create_function_chat/create_function_chat.py \
"call Create function with name='create_symbols_from_json_string' description=\"with schematic_file_in, schematic_file_out, symbols_string as parameter do following, import json; import skip; symbols = json.loads(symbols_string);  sch=skip.Schematic(schematic_file_in); for symbol_idx, symbol in enumerate(symbols):;    print(symbol);  labels_len=len(symbol[1]); sch_symbol=sch.symbol.reference_matches('JSOME'+str(labels_len)+'_1')[0].clone();    sch_symbol.setAllReferences(symbol[0]);    sch_symbol.property.Footprint.setValue(symbol[2]);   symbol_move=[20,0,0]; sch_symbol.move([sch_symbol.at.value[0]+symbol_move[0]*(symbol_idx+1), sch_symbol.at.value[1]+symbol_move[1]*(symbol_idx+1)], sch_symbol.at.value[2]+symbol_move[2]*(symbol_idx+1)); for label_idx, label in enumerate(symbol[1]):;    sch_label=sch.label.new();   sch_label.value=label;   pin_prefix='Pin_';  sch_label.move(sch_symbol.pin[pin_prefix+str(label_idx+1)].location.value); sch.write(schematic_file_out)\" use_cache=true" \
"call Create function with description='load json from json_file_name, and then output as json string without whitespaces' name='read_json_file' use_cache=true" \
'call read_json_file with json_file_name="freeeeg8-alpha_kicad_sch.json" with brief description, in pair of sentences' \
"call create_symbols_from_json_string with schematic_file_in=template_jsome.kicad_sch, schematic_file_out=freeeeg8-alpha.kicad_sch, symbols_string with all components from full response as JSON string format [ [ref1,[label11,label12],footprint1],[ref2,[label21,label22],footprint2] ]" \
"call Create function with name='kicad_sch_to_net_to_pcb' description=\"write 'import os'; then with kicad_sch_filename, net_filename, kicad_pcb_filename, pretty_folder_name as parameters do following, set environment variable KICAD7_FOOTPRINT_DIR=/usr/share/kicad/footprints/; execute in shell two commands, kicad-cli sch export netlist {kicad_sch_filename} --output {net_filename};  kinet2pcb -i {net_filename} --output {kicad_pcb_filename} -l {pretty_folder_name}\" use_cache=true" \
"call kicad_sch_to_net_to_pcb with kicad_sch_filename=freeeeg8-alpha.kicad_sch, net_filename=freeeeg8-alpha.net, kicad_pcb_filename=freeeeg8-alpha.kicad_pcb, pretty_folder_name=/content/freeeeg8-alpha" \
"/exit"

/content/freeeeg8-alpha
argument list: ['create_function_chat/create_function_chat.py', 'call Create function with name=\'create_symbols_from_json_string\' description="with schematic_file_in, schematic_file_out, symbols_string as parameter do following, import json; import skip; symbols = json.loads(symbols_string);  sch=skip.Schematic(schematic_file_in); for symbol_idx, symbol in enumerate(symbols):;    print(symbol);  labels_len=len(symbol[1]); sch_symbol=sch.symbol.reference_matches(\'JSOME\'+str(labels_len)+\'_1\')[0].clone();    sch_symbol.setAllReferences(symbol[0]);    sch_symbol.property.Footprint.setValue(symbol[2]);   symbol_move=[20,0,0]; sch_symbol.move([sch_symbol.at.value[0]+symbol_move[0]*(symbol_idx+1), sch_symbol.at.value[1]+symbol_move[1]*(symbol_idx+1)], sch_symbol.at.value[2]+symbol_move[2]*(symbol_idx+1)); for label_idx, label in enumerate(symbol[1]):;    sch_label=sch.label.new();   sch_label.value=label;   pin_prefix=\'Pin_\';  sch_label.move(sch_symbol.pin[pin_

In [ ]:
!pip install kigadgets
%cd /content/freeeeg8-alpha
!git clone https://github.com/atait/kicad-python.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 3.0 MB/s eta 0:00:00
/content/freeeeg8-alpha
Cloning into 'kicad-python'...
remote: Enumerating objects: 1712, done.
remote: Counting objects: 100% (393/393), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 1712 (delta 276), reused 339 (delta 240), pack-reused 1319 (from 1)
Receiving objects: 100% (1712/1712), 1.41 MiB | 3.43 MiB/s, done.
Resolving deltas: 100% (1138/1138), done.


In [ ]:
import pcbnew; print(pcbnew.__file__, pcbnew.SETTINGS_MANAGER.GetUserSettingsPath())

/usr/lib/python3/dist-packages/pcbnew.py /root/.config/kicad/7.0


In [ ]:
!link_kicad_python_to_pcbnew /usr/lib/python3/dist-packages/pcbnew.py /root/.config/kicad/7.0


1. Writing console startup script: for GUI snippet scripting
/root/.config/kicad/7.0/PyShell_pcbnew_startup.py
### Auto generated by kicad-python initialization for pcbnew console
import sys, pcbnew
sys.path.append("/usr/local/lib/python3.10/dist-packages")
import kicad
print('kigadgets (v{}) located at:'.format(kicad.__version__), kicad.__path__)
from kicad.pcbnew.board import Board
pcb = Board.from_editor()

2. Writing plugin importer: for action plugin development
/root/.config/kicad/7.0/scripting/plugins/initialize_kicad_python_plugin.py
### Auto generated by kicad-python initialization for pcbnew action plugins
import sys
sys.path.append("/usr/local/lib/python3.10/dist-packages")

3. Writing pcbnew path: for batch processing outside KiCAD
/usr/local/lib/python3.10/dist-packages/kicad/.path_to_pcbnew_module
/usr/lib/python3/dist-packages/pcbnew.py
Successfully linked kicad-python with pcbnew


In [ ]:
%cp template_jsome.kicad_pro freeeeg8-alpha.kicad_pro

In [ ]:
%cd /content/freeeeg8-alpha
!python create_function_chat/create_function_chat.py \
"call Create function with name='setup_pcb_from_string' description=\"with setup_string, pcb_size_string, board_file_name as parameters do following, import json; setup = json.loads(setup_string); import sys; sys.path.append('kicad-python/'); from kicad.pcbnew.board import Board; import pcbnew; pcb=Board.load(board_file_name); from pcbnew import FromMM; pcb_size = float(pcb_size_string); pcb.add_circle((0,0),pcb_size,'Edge.Cuts'); pcbnew.Refresh(); pcb.save(); for symbol in setup:;    print(symbol);    footprint=pcb.footprints[symbol[0]];  footprint.x=symbol[1];  footprint.y=symbol[2];  footprint.rotation = symbol[3];  if not(symbol[4] == footprint.layer) and (symbol[4] in ['F.Cu', 'B.Cu']):;    footprint._obj.Flip(footprint._obj.GetCenter(),True); pcbnew.Refresh(); pcb.save(); pcb=Board.from_editor(); board = pcb._obj; des = board.GetDesignSettings(); des.m_CopperEdgeClearance = pcbnew.FromMM(0.1); des.m_HoleClearance = FromMM(0.1); des.m_HoleToHoleMin = pcbnew.FromMM(0.2); des.m_MinClearance = FromMM(0.1); des.m_MinThroughDrill = pcbnew.FromMM(0.2); des.m_ViasMinSize = FromMM(0.3); net_set=des.m_NetSettings; def_net_class = net_set.m_DefaultNetClass; def_net_class.SetTrackWidth(FromMM(0.127)); def_net_class.SetClearance(FromMM(0.1)); def_net_class.SetViaDiameter(FromMM(0.3)); def_net_class.SetViaDrill(FromMM(0.2)); pcb.save();\" use_cache=true" \
"call Create function with description='load json from json_file_name, and then output as json string without whitespaces' name='read_json_file' use_cache=true" \
"call function create_function with parameters name='setup_pcb_layers', description=\"with gnd_size_mm_string, gnd_netname, board_file_name as parameters do following, import sys; sys.path.append('kicad-python/'); from kicad.pcbnew.board import Board; import pcbnew; pcb=Board.load(board_file_name); board = pcb._obj; enabled_layers =  board.GetEnabledLayers(); enabled_layers.AddLayer(pcbnew.In1_Cu); enabled_layers.AddLayer(pcbnew.In2_Cu); board.SetEnabledLayers(enabled_layers); from pcbnew import FromMM; gnd_size = FromMM(float(gnd_size_mm_string)); pts = [(-gnd_size, -gnd_size), (-gnd_size, gnd_size), (gnd_size, gnd_size), (gnd_size,-gnd_size)]; sps = pcbnew.SHAPE_POLY_SET(); chain = pcbnew.SHAPE_LINE_CHAIN(); for (x,y) in pts:;    chain.Append(x, y); chain.SetClosed(True); sps.AddOutline(chain); ps = pcbnew.PCB_SHAPE(board, pcbnew.SHAPE_T_POLY); ps.SetPolyShape(sps); z = pcbnew.ZONE(board); z.SetLayer(pcbnew.In2_Cu); z.AddPolygon( chain ); board.Add(z); filler = pcbnew.ZONE_FILLER(board); filler.Fill(board.Zones()); pcbnew.Refresh(); pcb.save(); gnd_net = None; nets=board.GetNetsByName(); for netname, net in nets.items():;    netname = net.GetNetname();    if netname == gnd_netname :;        gnd_net=net; pcbnew.Refresh(); z.SetNet(gnd_net); z.SetLocalClearance(FromMM(0.1)); pcbnew.Refresh(); pcb.save();\", use_cache=true" \
"call Create function with name='freerouting_and_pcbway' description=\"with freerouting_plugin, pcbway_plugin as parameters do following, import pcbnew; if freerouting_plugin=true:;        pcbnew.KICAD_PLUGINS['app_freerouting_kicad-plugin']['ModuleName'].plugin.FreeroutingPlugin().Run();  if pcbway_plugin=true:;         pcbnew.KICAD_PLUGINS['com_github_pcbway_PCBWay-Plug-in-for-Kicad']['ModuleName'].plugin.Run();\" use_cache=true" \
'call read_json_file with json_file_name="freeeeg8-alpha_kicad_pcb.json" with brief description, in pair of sentences' \
'call setup_pcb_from_string with setup_string with all components from full response as JSON string format [ [ref1,x1,y1,rotation1,layer1], [ref2,x2,y2,rotation2,layer2] ]; pcb_size_string="13", board_file_name="freeeeg8-alpha.kicad_pcb"' \
'call setup_pcb_layers with gnd_netname="/GND", gnd_size_mm_string="13", board_file_name="freeeeg8-alpha.kicad_pcb"' \
'call freerouting_and_pcbway with freerouting_plugin=false, pcbway_plugin=false' \
"/exit"


/content/freeeeg8-alpha
argument list: ['create_function_chat/create_function_chat.py', 'call Create function with name=\'setup_pcb_from_string\' description="with setup_string, pcb_size_string, board_file_name as parameters do following, import json; setup = json.loads(setup_string); import sys; sys.path.append(\'kicad-python/\'); from kicad.pcbnew.board import Board; import pcbnew; pcb=Board.load(board_file_name); from pcbnew import FromMM; pcb_size = float(pcb_size_string); pcb.add_circle((0,0),pcb_size,\'Edge.Cuts\'); pcbnew.Refresh(); pcb.save(); for symbol in setup:;    print(symbol);    footprint=pcb.footprints[symbol[0]];  footprint.x=symbol[1];  footprint.y=symbol[2];  footprint.rotation = symbol[3];  if not(symbol[4] == footprint.layer) and (symbol[4] in [\'F.Cu\', \'B.Cu\']):;    footprint._obj.Flip(footprint._obj.GetCenter(),True); pcbnew.Refresh(); pcb.save(); pcb=Board.from_editor(); board = pcb._obj; des = board.GetDesignSettings(); des.m_CopperEdgeClearance = pcbnew.Fro